In [1]:
import os
import pandas as pd

import pysentiment2 as ps

from dotenv import load_dotenv, find_dotenv
from rich import print

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, pipeline
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


import nltk
from nltk.corpus import stopwords
import spacy
from transformers import BertTokenizer, BertForSequenceClassification
from nltk.corpus import stopwords
from textblob import TextBlob
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%load_ext rich

load_dotenv(find_dotenv())

/var/folders/3s/3k3nlxhn13nfv9vncl9g9kbr0000gn/T/ipykernel_11752/2063040873.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/aladelca/anaconda3/envs/text_group/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [ ]:
nltk.download('stopwords')

### Load documents

In [4]:
documents_list = os.listdir("./extracted/")

docs = {
    doc.split("_")[0]: open(f"./extracted/{doc}", "r").read()
    for doc in documents_list
    if doc.endswith(".txt")
}


### Preprocess documents

In [5]:
print(docs['AAPL'])

Item 7. Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations 

The following discussion should be read in conjunction with the consolidated financial statements and accompanying 
notes included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2020 and 
2019 items and year-to-year comparisons between 2020 and 2019. Discussions of 2018 items and year-to-year 
comparisons between 2019 and 2018 that are not included in this Form 10-K can be found in &#8220;Management&#8217;s
Discussion and Analysis of Financial Condition and Results of Operations&#8221; in Part II, Item 7 of the 
Company&#8217;s Annual Report on Form 10-K for the fiscal year ended September 28, 2019. 

Fiscal Year Highlights 

COVID-19 Update 

COVID-19 has spread rapidly throughout the world, prompting governments and businesses to take unprecedented 
measures in response. Such measures have included restrictions on travel and business operations, temporary 
closures of businesses, and quarantines and shelter-in-place orders. The COVID-19 pandemic has significantly 
curtailed global economic activity and caused significant volatility and disruption in global financial markets. 
The COVID-19 pandemic and the measures taken by many countries in response have adversely affected and could in the
future materially adversely impact the Company&#8217;s business, results of operations, financial condition and 
stock price. 

During 2020, aspects of the Company&#8217;s business were adversely affected by the COVID-19 pandemic, with many of
the Company&#8217;s retail stores, as well as channel partner points of sale, temporarily closed at various times, 
and the vast majority of the Company&#8217;s employees working remotely. The Company has reopened some of its 
offices and the majority of its retail stores, subject to operating restrictions to protect public health and the 
health and safety of employees and customers, and it continues to work on safely re-opening the remainder of its 
offices and retail stores, subject to local rules and regulations. 

The full extent of the future impact of the COVID-19 pandemic on the Company&#8217;s operational and financial 
performance is currently uncertain and will depend on many factors outside the Company&#8217;s control, including, 
without limitation, the timing, extent, trajectory and duration of the pandemic, the development and availability 
of effective treatments and vaccines, the imposition of protective public safety measures, and the impact of the 
pandemic on the global economy and demand for consumer products. Refer to Part I, Item 1A of this Form 10-K under 
the heading &#8220;Risk Factors,&#8221; for more information. 

The Company believes its existing balances of cash, cash equivalents and marketable securities, along with 
commercial paper and other short-term liquidity arrangements, will be sufficient to satisfy its working capital 
needs, capital asset purchases, dividends, share repurchases, debt repayments and other liquidity requirements 
associated with its existing operations. 

Fiscal 2020 Highlights 

Total net sales increased 6% or $14.3 billion during 2020 compared to 2019, primarily driven by higher net sales of
Services and Wearables, Home and Accessories. The weakness in foreign currencies had an unfavorable impact on net 
sales during 2020. 

In April 2020, the Company announced an increase to its current share repurchase program authorization from $175 
billion to $225 billion and raised its quarterly dividend from $0.1925 to $0.205 per share beginning in May 2020. 
During 2020, the Company repurchased $72.5 billion of its common stock and paid dividends and dividend equivalents 
of $14.1 billion. 

On August 28, 2020, the Company effected a four-for-one stock split to shareholders of record as of August 24, 
2020. All share, RSU and per share or per RSU information has been retroactively adjusted to reflect the stock

## Preprocessing

In [77]:
from nltk.stem import WordNetLemmatizer
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('&#',' ')
    text = text.replace("\\ ",'')
    text = text.lower()
    return text

def text_tokenize(text):
    nltk_tokens_no_stopwords = nltk.word_tokenize(text)
    return nltk_tokens_no_stopwords

def remove_stopwords(text):
    nltk_tokens_no_stopwords = [word for word in text if word not in stopwords.words('english')]
    text_no_stopwords = ' '.join(nltk_tokens_no_stopwords)
    return text_no_stopwords

def lemmatize_text(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    text_lemmatized = ' '.join([token.lemma_ for token in doc])
    return text_lemmatized

def preprocessing_text(text):
    text = clean_text(text)
    text = text_tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

## Sentiment analysis

### Loughran and McDonald Financial Sentiment Dictionaries

In [44]:
lm = ps.LM()

In [5]:
for k, v in docs.items():
    print(
        f"[bold green]Sentiment for {k}[/bold green]\n",
        lm.get_score(lm.tokenize(v)),
    )


Sentiment for AMZN

{'Positive': 108, 'Negative': 106, 'Polarity': 0.009345794348851428, 'Subjectivity': 0.05423213379264264}

Sentiment for META

{'Positive': 122, 'Negative': 162, 'Polarity': -0.14084506992660187, 'Subjectivity': 0.05527442583587496}

Sentiment for NFLX

{'Positive': 53, 'Negative': 117, 'Polarity': -0.3764705860207613, 'Subjectivity': 0.06348020908757274}

Sentiment for MSFT

{'Positive': 103, 'Negative': 107, 'Polarity': -0.0190476189569161, 'Subjectivity': 0.03885291396135931}

Sentiment for AAPL

{'Positive': 41, 'Negative': 89, 'Polarity': -0.3692307663905326, 'Subjectivity': 0.05256773147894552}

In [80]:

for k, v in docs.items():
    print(
        f"[bold green]Sentiment for {k}[/bold green]\n",
        lm.get_score(lm.tokenize(preprocessing_text(v))),
    )
 



Sentiment for AMZN

{'Positive': 106, 'Negative': 106, 'Polarity': 0.0, 'Subjectivity': 0.05469556242138918}

Sentiment for META

{'Positive': 122, 'Negative': 162, 'Polarity': -0.14084506992660187, 'Subjectivity': 0.05650616791553797}

Sentiment for NFLX

{'Positive': 53, 'Negative': 117, 'Polarity': -0.3764705860207613, 'Subjectivity': 0.06508422662133069}

Sentiment for MSFT

{'Positive': 103, 'Negative': 107, 'Polarity': -0.0190476189569161, 'Subjectivity': 0.03952569169216531}

Sentiment for AAPL

{'Positive': 40, 'Negative': 89, 'Polarity': -0.3798449582957755, 'Subjectivity': 0.0547538200107157}

### Using `TextBlob`

In [81]:
for k, v in docs.items():
    print(
        f"[bold green]Sentiment for {k}[/bold green]\n",
        TextBlob(preprocessing_text(v)).sentiment
    )
 


Sentiment for AMZN

Sentiment(polarity=0.05294354458230046, subjectivity=0.34246241840739516)

Sentiment for META

Sentiment(polarity=0.007454579853724121, subjectivity=0.4104204597358634)

Sentiment for NFLX

Sentiment(polarity=0.036569709211097375, subjectivity=0.39810747093523413)

Sentiment for MSFT

Sentiment(polarity=0.043081127206127195, subjectivity=0.32242090815167823)

Sentiment for AAPL

Sentiment(polarity=0.04445166745055878, subjectivity=0.32311038062701025)

### Using VADER

In [82]:
vader = SentimentIntensityAnalyzer()
for k, v in docs.items():
    print(
        f"[bold green]Sentiment for {k}[/bold green]\n",
        vader.polarity_scores(preprocessing_text(v))
    )




Sentiment for AMZN

{'neg': 0.04, 'neu': 0.807, 'pos': 0.153, 'compound': 1.0}

Sentiment for META

{'neg': 0.046, 'neu': 0.804, 'pos': 0.15, 'compound': 1.0}

Sentiment for NFLX

{'neg': 0.058, 'neu': 0.786, 'pos': 0.156, 'compound': 0.9999}

Sentiment for MSFT

{'neg': 0.043, 'neu': 0.803, 'pos': 0.154, 'compound': 1.0}

Sentiment for AAPL

{'neg': 0.069, 'neu': 0.812, 'pos': 0.119, 'compound': 0.9995}

### Using pre-trained model

@misc{yang2020finbert,
    title={FinBERT: A Pretrained Language Model for Financial Communications},
    author={Yi Yang and Mark Christopher Siy UY and Allen Huang},
    year={2020},
    eprint={2006.08097},
    archivePrefix={arXiv},
    }

Only allows 512 tokens

In [67]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

config.json: 100%|██████████| 533/533 [00:00<00:00, 480kB/s]
pytorch_model.bin: 100%|██████████| 439M/439M [00:43<00:00, 10.1MB/s] 
vocab.txt: 100%|██████████| 226k/226k [00:00<00:00, 8.97MB/s]


In [84]:


def get_predictions(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]

    labels = {0:'neutral', 1:'positive',2:'negative'}

    return (text, '----', labels[np.argmax(outputs.detach().numpy())])

for k, v in docs.items():
    print(
        f"[bold green]Sentiment for {k}[/bold green]\n",
        get_predictions(preprocessing_text(v)[:512])
    )

Sentiment for AMZN

(
    'item 7 . management 8217 ; discussion analysis financial condition result operation # # table_end forward - 
look statement annual report form 10 - k include forward - look statement within mean private security litigation 
reform act 1995 . statement statement historical fact , include statement regard guidance , industry prospect , 
future result operation financial position , make annual report form 10 - k forward - look . use word anticipate , 
believe , expect , future , intend , similar expression identif',
    '----',
    'neutral'
)

Sentiment for META

(
    "item 7 . management 's discussion analysis financial condition result operation read follow discussion 
financial condition result operation conjunction consolidate financial statement related note include part ii , 
item 8 , ` ` financial statement supplementary datum '' annual report form 10 - k. addition historical consolidated
financial information , follow discussion contain forward - look statement reflect plan , estimate , belief . 
actual result could differ materially discuss forward - look statement ",
    '----',
    'neutral'
)

Sentiment for NFLX

(
    "item 7 . management 8217 ; discussion analysis financial condition result operation section form 10 - k 
generally discuss 2020 2019 item year - to - year comparison 2020 2019 . discussion 2018 item year - to - year 
comparison 2019 2018 include form 10 - k find ` ` management 's discussion analysis financial condition result 
operation '' part ii , item 7 company 's annual report form 10 - k fiscal year end december 31 , 2019 . result 
operation follow represent consolidated performance highlight : # # table_s",
    '----',
    'neutral'
)

Sentiment for MSFT

(
    'item 7 . management 8217 ; discussion analysis financial condition result operation follow management 8217 ; 
discussion analysis financial condition result operation ( 8220 ; md 38 ; 8221 ; ) intend help reader understand 
result operation financial condition microsoft corporation . md 38 ; provide supplement , read conjunction , 
consolidated financial statement accompany note financial statement ( part ii , item 8 form 10 - k ) . overview 
microsoft technology company whose mission empower every person every',
    '----',
    'neutral'
)

Sentiment for AAPL

(
    'item 7 . management 8217 ; discussion analysis financial condition result operation follow discussion read 
conjunction consolidate financial statement accompany note include part ii , item 8 form 10 - k. section form 10 - 
k generally discuss 2020 2019 item year - to - year comparison 2020 2019 . discussion 2018 item year - to - year 
comparison 2019 2018 include form 10 - k find 8220 ; management 8217 ; discussion analysis financial condition 
result operation 8221 ; part ii , item 7 company 8217 ; annual rep',
    '----',
    'neutral'
)